# Tarefa 2 — GBM (Gradient Boosting Machine)

## 1) Cite 5 diferenças entre o AdaBoost e o GBM

1. **Forma de corrigir erros**
   - **AdaBoost:** repondera as amostras, aumentando o peso das observações erradas para o próximo modelo focar nelas.
   - **GBM:** ajusta o próximo modelo para aproximar os **resíduos/gradiente do erro**, ou seja, “corrige” diretamente o erro do modelo atual.

2. **Função de perda (loss)**
   - **AdaBoost:** é mais “fixo” na formulação clássica (focado em classificação) e segue uma lógica específica de penalização.
   - **GBM:** é mais geral e trabalha com diferentes **funções de perda**, o que facilita adaptação para classificação e regressão.

3. **Como os modelos são somados**
   - **AdaBoost:** combina modelos com pesos baseados no desempenho de cada rodada.
   - **GBM:** combina modelos em uma soma incremental controlada por **learning_rate**, seguindo a direção do gradiente.

4. **Modelos base típicos**
   - **AdaBoost:** geralmente usa **stumps** (árvores muito rasas, como depth=1).
   - **GBM:** normalmente usa **árvores rasas**, mas costuma precisar de mais controle de profundidade/folhas para evitar overfitting.

5. **Sensibilidade a ruído/outliers**
   - **AdaBoost:** tende a ser mais sensível porque repondera muito as amostras difíceis/erradas.
   - **GBM:** também pode sofrer com overfitting, mas possui mais “alavancas” de regularização (subsample, max_depth, min_samples_leaf etc.) e maior flexibilidade de loss.
----


## 2) Exemplo de GBM em Python (Scikit-learn) — Classificação e Regressão

Nesta parte, serão reproduzidos exemplos com **GradientBoostingClassifier** e **GradientBoostingRegressor**, seguindo a lógica apresentada na documentação do Scikit-learn: 
- carregar dataset
- separar treino/teste
- treinar o modelo
- avaliar resultados

Serão usados datasets prontos do `sklearn`, para evitar dependência de download externo.


In [10]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

from sklearn.ensemble import GradientBoostingClassifier, GradientBoostingRegressor
from sklearn.datasets import load_iris, load_diabetes


### 2.1) Exemplo de Classificação — GradientBoostingClassifier (load_iris)


In [13]:
data_cls = load_iris(as_frame=True)
df_cls = data_cls.frame.copy()

print("Shape do df (classificação):", df_cls.shape)
print("Colunas:", df_cls.columns.tolist())

df_cls.head()


Shape do df (classificação): (150, 5)
Colunas: ['sepal length (cm)', 'sepal width (cm)', 'petal length (cm)', 'petal width (cm)', 'target']


,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


In [14]:
target_cls = "target"
X_cls = df_cls.drop(columns=[target_cls])
y_cls = df_cls[target_cls]

X_train_cls, X_test_cls, y_train_cls, y_test_cls = train_test_split(
    X_cls, y_cls, test_size=0.30, random_state=42, stratify=y_cls
)

print("Treino (cls):", X_train_cls.shape, "Teste (cls):", X_test_cls.shape)


Treino (cls): (105, 4) Teste (cls): (45, 4)


In [15]:
gbc = GradientBoostingClassifier(
    n_estimators=200,
    learning_rate=0.1,
    max_depth=3,   # profundidade das árvores base (controla complexidade)
    random_state=42
)

gbc.fit(X_train_cls, y_train_cls)
y_pred_cls = gbc.predict(X_test_cls)

acc = accuracy_score(y_test_cls, y_pred_cls)
print(f"Acurácia (GBM Classificação): {acc:.4f}")


Acurácia (GBM Classificação): 0.9111


In [16]:
print("\nClassification Report:\n", classification_report(y_test_cls, y_pred_cls))
print("\nMatriz de confusão:\n", confusion_matrix(y_test_cls, y_pred_cls))



Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00        15
           1       0.82      0.93      0.88        15
           2       0.92      0.80      0.86        15

    accuracy                           0.91        45
   macro avg       0.92      0.91      0.91        45
weighted avg       0.92      0.91      0.91        45


Matriz de confusão:
 [[15  0  0]
 [ 0 14  1]
 [ 0  3 12]]


In [17]:
res_cls = pd.DataFrame({
    "y_real": y_test_cls.reset_index(drop=True),
    "y_pred": pd.Series(y_pred_cls)
})

print("Linhas no resumo (cls):", len(res_cls))
res_cls.head()


Linhas no resumo (cls): 45


,y_real,y_pred
0,2,2
1,1,1
2,2,1
3,1,1
4,2,1


### 2.2) Exemplo de Regressão — GradientBoostingRegressor (load_diabetes)


In [18]:
data_reg = load_diabetes(as_frame=True)
df_reg = data_reg.frame.copy()

print("Shape do df (regressão):", df_reg.shape)
print("Colunas:", df_reg.columns.tolist())

df_reg.head()


Shape do df (regressão): (442, 11)
Colunas: ['age', 'sex', 'bmi', 'bp', 's1', 's2', 's3', 's4', 's5', 's6', 'target']


,age,sex,bmi,bp,s1,s2,s3,s4,s5,s6,target
0,0.038076,0.050680,0.061696,0.021872,-0.044223,-0.034821,-0.043401,-0.002592,0.019907,-0.017646,151.0
1,-0.001882,-0.044642,-0.051474,-0.026328,-0.008449,-0.019163,0.074412,-0.039493,-0.068332,-0.092204,75.0
2,0.085299,0.050680,0.044451,-0.005670,-0.045599,-0.034194,-0.032356,-0.002592,0.002861,-0.025930,141.0
3,-0.089063,-0.044642,-0.011595,-0.036656,0.012191,0.024991,-0.036038,0.034309,0.022688,-0.009362,206.0
4,0.005383,-0.044642,-0.036385,0.021872,0.003935,0.015596,0.008142,-0.002592,-0.031988,-0.046641,135.0


In [19]:
target_reg = "target"
X_reg = df_reg.drop(columns=[target_reg])
y_reg = df_reg[target_reg]

X_train_reg, X_test_reg, y_train_reg, y_test_reg = train_test_split(
    X_reg, y_reg, test_size=0.30, random_state=42
)

print("Treino (reg):", X_train_reg.shape, "Teste (reg):", X_test_reg.shape)


Treino (reg): (309, 10) Teste (reg): (133, 10)


In [20]:
gbr = GradientBoostingRegressor(
    n_estimators=300,
    learning_rate=0.05,
    max_depth=3,
    random_state=42
)

gbr.fit(X_train_reg, y_train_reg)
y_pred_reg = gbr.predict(X_test_reg)

rmse = np.sqrt(mean_squared_error(y_test_reg, y_pred_reg))
mae = mean_absolute_error(y_test_reg, y_pred_reg)
r2 = r2_score(y_test_reg, y_pred_reg)

print(f"RMSE (GBM Regressão): {rmse:.4f}")
print(f"MAE  (GBM Regressão): {mae:.4f}")
print(f"R²   (GBM Regressão): {r2:.4f}")


RMSE (GBM Regressão): 55.9338
MAE  (GBM Regressão): 44.7004
R²   (GBM Regressão): 0.4204


In [21]:
res_reg = pd.DataFrame({
    "y_real": y_test_reg.reset_index(drop=True),
    "y_pred": pd.Series(y_pred_reg)
})

print("Linhas no resumo (reg):", len(res_reg))
res_reg.head()


Linhas no resumo (reg): 133


,y_real,y_pred
0,219.0,180.014829
1,70.0,184.700515
2,202.0,153.095774
3,230.0,280.543635
4,111.0,119.757857


## 3) Cite 5 hiperparâmetros importantes no GBM

1. **n_estimators**  
   Quantidade de árvores (iterações). Mais árvores podem melhorar desempenho, mas aumentam custo e risco de overfitting.

2. **learning_rate**  
   Controla o tamanho do passo a cada iteração (quanto cada nova árvore contribui). Valores menores costumam exigir mais estimators.

3. **max_depth** (ou parâmetros equivalentes de complexidade da árvore)  
   Controla a complexidade das árvores base. Árvores mais profundas capturam mais padrões, mas aumentam risco de overfitting.

4. **subsample**  
   Fração das amostras usada para treinar cada árvore. Quando < 1.0, vira uma forma de regularização e aproxima o **Stochastic Gradient Boosting**.

5. **min_samples_leaf** (ou min_samples_split)  
   Controla o mínimo de amostras em folhas (ou para dividir nós), reduzindo árvores muito específicas e ajudando na generalização.


## 4) GridSearch para melhores hiperparâmetros (opcional)

A seguir, aplicamos `GridSearchCV` para testar combinações de hiperparâmetros e buscar melhor desempenho.
(Como já executei a etapa, ela entra como complemento do notebook.)


In [22]:
from sklearn.model_selection import GridSearchCV

param_grid_cls = {
    "n_estimators": [100, 200, 400],
    "learning_rate": [0.01, 0.05, 0.1],
    "max_depth": [1, 2, 3]
}

gs_cls = GridSearchCV(
    GradientBoostingClassifier(random_state=42),
    param_grid=param_grid_cls,
    scoring="accuracy",
    cv=5,
    n_jobs=-1
)

gs_cls.fit(X_train_cls, y_train_cls)

print("Melhores parâmetros (cls):", gs_cls.best_params_)
print("Melhor score CV (cls):", gs_cls.best_score_)

best_cls = gs_cls.best_estimator_
y_pred_best_cls = best_cls.predict(X_test_cls)

print("Acurácia no teste (cls):", accuracy_score(y_test_cls, y_pred_best_cls))


Melhores parâmetros (cls): {'learning_rate': 0.01, 'max_depth': 1, 'n_estimators': 100}
Melhor score CV (cls): 0.9523809523809523
Acurácia no teste (cls): 0.9333333333333333


In [23]:
param_grid_reg = {
    "n_estimators": [100, 300, 500],
    "learning_rate": [0.01, 0.05, 0.1],
    "max_depth": [2, 3, 4],
    "subsample": [0.6, 0.8, 1.0]
}

gs_reg = GridSearchCV(
    GradientBoostingRegressor(random_state=42),
    param_grid=param_grid_reg,
    scoring="neg_root_mean_squared_error",
    cv=5,
    n_jobs=-1
)

gs_reg.fit(X_train_reg, y_train_reg)

print("Melhores parâmetros (reg):", gs_reg.best_params_)
print("Melhor score CV (reg):", gs_reg.best_score_)

best_reg = gs_reg.best_estimator_
y_pred_best_reg = best_reg.predict(X_test_reg)

rmse_best = np.sqrt(mean_squared_error(y_test_reg, y_pred_best_reg))
print("RMSE no teste (reg):", rmse_best)


Melhores parâmetros (reg): {'learning_rate': 0.01, 'max_depth': 2, 'n_estimators': 300, 'subsample': 0.6}
Melhor score CV (reg): -58.396274499821494
RMSE no teste (reg): 52.1936981862119


## 5) Stochastic GBM vs GBM tradicional (Friedman): qual a maior diferença?

A maior diferença é que o **Stochastic GBM** introduz **amostragem aleatória de observações** a cada iteração (por exemplo, usando `subsample < 1.0`).  
Isso faz com que cada árvore seja treinada em apenas uma parte dos dados, adicionando aleatoriedade e funcionando como regularização, o que tende a reduzir overfitting e pode melhorar generalização.

Já o GBM tradicional usa o conjunto completo de treino em todas as iterações (equivalente a `subsample = 1.0`).
